# Importing necessary package

In [1]:
import pandas as pd 
import numpy as np
import time

from scipy.stats import norm
from scipy.optimize import minimize
from scipy.integrate import quad

import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

np.set_printoptions(suppress=True)

from __Function__ import *

np.random.seed(42)

# Importing Data

In [2]:
path = r'C:\Users\baris\Desktop\Master Thesis\1. Python\Data\Option_Calibration_SPX_18072025.csv'
df = pd.read_csv(path, index_col=None)
spx_price = 6296.79

# Calibration of the Heston CF Model

In [ ]:
def heston_charfunc(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):

    a = kappa*theta
    b = kappa+lambd

    rspi = rho*sigma*phi*1j
    d = np.sqrt((rho*sigma*phi*1j - b)**2 + (phi*1j+phi**2)*sigma**2)

    g = (b-rspi+d) / (b-rspi-d)
    exp1 = np.exp(r*phi*1j*tau)
    term2 = S0**(phi*1j) * ((1-g*np.exp(d*tau))/(1-g))**(-2*a/sigma**2)

    max_exponent = 700
    exp_value = a * tau * (b - rspi + d) / sigma**2 + v0 * (b - rspi + d) * ((1 - np.exp(d * tau)) / (1 - g * np.exp(d * tau))) / sigma**2
    exp_value = np.clip(exp_value, -max_exponent, max_exponent)
    exp2 = np.exp(exp_value)
    
    return exp1*term2*exp2

In [ ]:
def integrand(phi, S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
    numerator = np.exp(r*tau)*heston_charfunc(phi-1j, *args) - K*heston_charfunc(phi, *args)
    denominator = 1j*phi*K**(1j*phi)
    
    return numerator/denominator

In [ ]:
def heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):

    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
    P, umax, N = 0, 100, 10000
    dphi=umax/N 

    for i in range(1,N):
        phi = dphi * (2*i + 1)/2
        numerator = np.exp(r*tau) * heston_charfunc(phi-1j, *args) - K * heston_charfunc(phi, *args)
        denominator = 1j * phi * K**(1j * phi)
        P += dphi * numerator / denominator

    return np.real((S0 - K * np.exp(-r * tau)) / 2 + P/np.pi)

In [ ]:
def heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):

    args = (S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)
    real_integral, err = np.real(quad(integrand, 0, 100, args=args))
    
    return (S0 - K*np.exp(-r*tau))/2 + real_integral/np.pi

In [7]:
S0 = spx_price
r = 0.0408

r = np.array([0.0408 for i in range(len(df))])
K = df['Strike'].to_numpy('float')
maturity = df['Maturity'].to_numpy('float')
market_price = df['Price'].to_numpy('float')

params = {
    "v0": {"x0": 0.021, "lbub": [1e-4, 0.5]},
    "kappa": {"x0": 1.5, "lbub": [1e-3, 5]},
    "theta": {"x0": 0.02, "lbub": [1e-4, 0.5]},
    "sigma": {"x0": 0.3, "lbub": [1e-2, 1]},
    "rho": {"x0": -0.5, "lbub": [-1, 1]},
    "lambd": {"x0": -0.5, "lbub": [-1, 1]},
    }

In [11]:
def SqErr(x):
    v0, kappa, theta, sigma, rho, lambd = x
    err = np.sum((market_price - heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, maturity, r))**2 / len(market_price))
    return err + 0

In [12]:
x0 = [param["x0"] for key, param in params.items()]
bnds = [param["lbub"] for key, param in params.items()]

In [13]:
start = time.time()
result = minimize(SqErr, x0, method='L-BFGS-B', options={'maxiter': 1e6, 'ftol': 1e-5, 'disp':True}, bounds=bnds)
end = time.time()

total_time = end - start

result

  message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
  success: True
   status: 0
      fun: 62.266366831041864
        x: [ 2.028e-02  2.133e+00  1.730e-02  1.146e-01 -2.038e-01
            -2.078e-01]
      nit: 29
      jac: [-5.802e+01 -3.685e-01 -3.391e+01 -5.795e-01  1.829e+00
            -9.317e-02]
     nfev: 238
     njev: 34
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>

In [14]:
total_time

246.62200665473938

In [15]:
v0, kappa, theta, sigma, rho, lambd = [param for param in result.x]
v0, kappa, theta, sigma, rho, lambd 

(np.float64(0.020278187219256612),
 np.float64(2.1329444106531006),
 np.float64(0.017297302604897306),
 np.float64(0.11457520850858496),
 np.float64(-0.2038406618309211),
 np.float64(-0.20777167036522193))

In [16]:
heston_prices = heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, maturity, r)

In [17]:
df['Heston_CF'] = heston_prices
df['Error'] = abs((df['Heston_CF'] - market_price))

In [18]:
df['Error'].mean(), df['Error'].median()

(np.float64(6.305833692132371), np.float64(4.854343741332542))

# Summary of the Heston CF 

In [19]:
path = r'C:\Users\baris\Desktop\Master Thesis\1. Python\Data\Option_Data_SPX_18072025.csv'
df_pricing = pd.read_csv(path, index_col=None)

In [20]:
r = np.array([0.0408 for i in range(len(df_pricing))])
K = df_pricing['Strike'].to_numpy('float')
maturity = df_pricing['Maturity'].to_numpy('float')
market_prices = df_pricing['Price'].to_numpy('float')

In [21]:
heston_prices = heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, maturity, r)

In [22]:
df_pricing['Heston_CF'] = heston_prices
df_pricing['Error'] = abs((heston_prices - market_prices))

In [23]:
maturity_error = Comparaison.comparaison_maturity(df_pricing)
maturity_error

,Metric Maturity,Mean Eror (abs)
0,"Mean (Filtered for maturity < 0.3, Price = 143...",6.7269
1,"Mean (Filtered for maturity >= 0.3 and < 0.6, ...",6.2575
2,"Mean (Filtered for maturity >= 0.6, Price = 35...",6.2137


In [24]:
strike_error = Comparaison.comparaison_strike(df_pricing, spx_price)
strike_error

,Metric Strike,Mean Eror (abs)
0,"Mean (Filtered for strike < 100, Price = 357.6...",9.2830
1,"Mean (Filtered for strike >= 100 and < 105, Pr...",5.6573
2,"Mean (Filtered for strike >= 105, Price = 75.9...",3.5780


In [25]:
surface_error = Comparaison.comparaison_surface(df_pricing)
surface_error

,Metric Surface,Mean Eror (abs)
0,"Mean (Filtered for price < 81.70$, 20% Quantile)",5.1794
1,"Mean (Filtered for price > 81.70$, 20% Quantile)",6.7500
2,"Mean (Filtered for price > 214.65$, 50% Quantile)",8.2757
3,"Mean (Filtered for price > 370.98$, 80% Quantile)",9.3834
4,Mean,6.4459
5,Median,5.1133


In [26]:
Comparaison.saving_data(maturity_error, 'Heston\Heston_maturity_error')
Comparaison.saving_data(strike_error, 'Heston\Heston_strike_error')
Comparaison.saving_data(surface_error, 'Heston\Heston_surface_error')

# Implied Volatility

In [27]:
def bs_price_call(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

def bs_vega(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

def implied_vol_newton(price, S, K, T, r, tol=1e-8, max_iterations=100):
    sigma = 0.2 # Initial guess
    for i in range(max_iterations):
        price_bs = bs_price_call(S, K, T, r, sigma)
        vega = bs_vega(S, K, T, r, sigma)
        price_diff = price_bs - price
        if abs(price_diff) < tol:
            return sigma
        sigma -= price_diff / vega
        sigma = max(sigma, 1e-8) # Security to avoid 0
    return sigma #If no convergence, return the last value

In [28]:
IV_Heston = []
r = 0.0408

for price, strike, maturity in zip(df_pricing['Heston_CF'], df_pricing['Strike'], df_pricing['Maturity']):
    IV_Heston.append(implied_vol_newton(price, spx_price, strike, maturity, r))

df_pricing['IV_Heston'] = IV_Heston

In [29]:
df_pricing

,Maturity,Strike,Price,IV,Heston_CF,Error,IV_Heston
0,0.109514,6000,370.98,0.197277,354.650981,16.329019,0.167672
1,0.109514,6150,247.82,0.176047,231.463041,16.356959,0.153242
2,0.109514,6175,212.64,0.150893,213.054012,0.414012,0.151459
3,0.109514,6200,196.59,0.151420,195.358157,1.231843,0.149798
4,0.109514,6225,177.37,0.146910,178.409238,1.039238,0.148239
...,...,...,...,...,...,...,...
356,0.742466,6650,238.40,0.141415,231.822835,6.577165,0.138346
357,0.742466,6700,206.71,0.135285,207.927861,1.217861,0.135861
358,0.742466,6750,188.41,0.134846,185.330423,3.079577,0.133365
359,0.742466,6800,165.62,0.131625,164.017326,1.602674,0.130837


In [30]:
import plotly.graph_objs as go

# Filtrer les lignes où IV ou IV_Heston est égal à zéro
df_plot = df_pricing[(df_pricing['IV'] > 0.001) & (df_pricing['IV_Heston'] > 0.001)]

fig = go.Figure()

# Market IV as Mesh
fig.add_trace(go.Mesh3d(x=df_plot['Maturity'], y=df_plot['Strike'], z=df_plot['IV'],
                        color='blue',
                        opacity=0.4,
                        name='Market IV'))

# Heston IV as Markers
fig.add_trace(go.Scatter3d(x=df_plot['Maturity'], y=df_plot['Strike'], z=df_plot['IV_Heston'],
                           mode='markers',
                           marker=dict(size=7, color='red'),
                           name='Heston IV'
                           ))

fig.update_layout(title_text="Market Implied Volatility Surface (Mesh) vs Calibrated Heston IV (Markers)",
                  scene=dict(xaxis_title='MATURITY', yaxis_title='STRIKE', zaxis_title='IMPLIED VOLATILITY'),
                  width=1200,
                  height=1000,
                  legend=dict(x=0.05, y=0.95))

fig.show()
